In [7]:
import retro
import time

#Troubleshooting
#!python --version
#%pip list
#pip install gym
#pip install gym-retro
#!python -m retro.import StreetFighterIISpecialChampionEdition-Genesis.md

In [10]:
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [9]:
env.close()

In [ ]:
#show env

obs = env.reset()
done = False
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        time.sleep(1/60)
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)

Exception ignored on calling ctypes callback function: <function Win32Window._get_window_proc.<locals>.f at 0x000001C8A7832550>
Traceback (most recent call last):
  File "C:\Users\Gaurav\anacondaWrapper\envs\Python38\lib\site-packages\pyglet\window\win32\__init__.py", line 738, in f
    def f(hwnd, msg, wParam, lParam):
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function Win32Window._get_window_proc.<locals>.f at 0x000001C8A7832550>
Traceback (most recent call last):
  File "C:\Users\Gaurav\anacondaWrapper\envs\Python38\lib\site-packages\pyglet\window\win32\__init__.py", line 738, in f
    def f(hwnd, msg, wParam, lParam):
KeyboardInterrupt: 


In [ ]:
from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2

In [ ]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(84,84,1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        
    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        
        reward = info['score'] - self.score
        self.score = info['score']
        
        return frame_delta, reward, done, info
    
    def render(self):
        self.game.render()
    
    def reset(self):
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        self.score = 0
        return obs
    
    def preprocess(self, observation):
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84,84), interpolation = cv2.INTER_CUBIC)
        channels = np.reshape(resize, (84,84,1))
        return channels
    
    def close(self):
        self.game.close()

In [ ]:
import optuna
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [ ]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [ ]:
def optimize_ppo(trial):
    return {
        'n_steps': trial.suggest_int('n_steps', 2048, 8192),
        'gamma': trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range': trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    } 

In [ ]:
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial)
        
        env = StreetFighter
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')
        
        model = PPO('CnnPolicy', env, tensorboard_log = LOG_DIR, verbose = 0, **model_params)
        model.learn(total_timesteps = 100000)
        
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()
        
        SAVE_PATH = os.path.join(OPT_DIR, 'tiral_{}_best_mmodel'.format(trial.number))
        model.save(SAVE_PATH)
        
        return mean_reward
    
    except Exception as e:
        return -1000